# Sequential Labelling Lora

## Import Modules

In [2]:
!pip install -q evaluate nusacrowd seqeval peft
import evaluate
import numpy as np
import transformers
import tensorflow as tf
import pandas as pd
import torch
import wandb

from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForTokenClassification, create_optimizer, TFAutoModelForTokenClassification, pipeline
from IPython.display import display, HTML
from transformers.keras_callbacks import KerasMetricCallback, PushToHubCallback

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Import Dataset

In [4]:
nergrit = load_dataset('NusaCrowd/nergrit')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [5]:
print(nergrit["train"][0])

{'index': '0', 'tokens': ['Indonesia', 'mengekspor', 'produk', 'industri', 'skala', 'besar', 'ke', 'Amerika', 'Serikat', '.', 'Ekspor', 'dilakukan', 'melalui', 'Pelabuhan', 'Tanjung', 'Priok', ',', 'Jakarta', 'Utara', ',', 'Selasa', '(', '15', '/', '8', '/', '2018', ')', '.', 'Komoditas', 'yang', 'dikirim', 'terdiri', 'dari', '50', 'persen', 'sepatu', ',', '15', 'persen', 'garmen', ',', '10', 'persen', 'produk', 'karet', ',', 'ban', 'dan', 'turunannya', ',', 'alat', '-', 'alat', 'elektronik', '10', 'persen', ',', 'dan', 'produk', 'lainnya', '15', 'persen', '.', '"', 'Bukan', 'bahan', 'mentah', ',', 'tetapi', 'sudah', 'bahan', '-', 'bahan', 'produksi', ',', 'produk', '-', 'produk', 'industri', 'yang', 'kita', 'harapkan', 'ini', 'akan', 'meningkatkan', 'ekspor', 'kita', ',', '"', 'kata', 'Presiden', 'dalam', 'sambutannya', 'pada', 'acara', 'pelepasan', 'ekspor', 'di', 'Jakarta', 'International', 'Container', 'Terminal', '(', 'JICT', ')', ',', 'Selasa', '(', '15', '/', '5', '/', '2018', '

In [6]:
# Extract the 'ner_tag' column from the training set
ner_tags_list = nergrit["train"]["ner_tag"]

# Flatten the list of lists
flattened_ner_tags = [tag for sublist in ner_tags_list for tag in sublist]

# Get the unique labels
unique_labels = list(set(flattened_ner_tags))

print(unique_labels)

['B-PRC', 'B-CRD', 'B-FAC', 'B-NOR', 'O', 'B-ORG', 'B-PRD', 'I-DAT', 'B-GPE', 'I-TIM', 'I-ORG', 'I-MON', 'I-PRC', 'I-LAW', 'I-LAN', 'I-ORD', 'I-PER', 'B-TIM', 'B-EVT', 'I-LOC', 'B-DAT', 'B-LAN', 'B-PER', 'I-QTY', 'I-CRD', 'I-FAC', 'I-REG', 'I-GPE', 'B-QTY', 'I-EVT', 'B-REG', 'B-WOA', 'I-WOA', 'B-LAW', 'B-MON', 'B-LOC', 'I-PRD', 'B-ORD', 'I-NOR']


Based on the documentation in https://huggingface.co/datasets/NusaCrowd/nergrit.
<br>
Label:

'CRD': Cardinal

'DAT': Date

'EVT': Event

'FAC': Facility

'GPE': Geopolitical Entity

'LAW': Law Entity (such as Undang-Undang)

'LOC': Location

'MON': Money

'NOR': Political Organization

'ORD': Ordinal

'ORG': Organization

'PER': Person

'PRC': Percent

'PRD': Product

'QTY': Quantity

'REG': Religion

'TIM': Time

'WOA': Work of Art

'LAN': Language

In [7]:
labels_dict = {
    'O': 0,
    'B-CRD': 1, 'I-CRD': 2,
    'B-DAT': 3, 'I-DAT': 4,
    'B-EVT': 5, 'I-EVT': 6,
    'B-FAC': 7, 'I-FAC': 8,
    'B-GPE': 9, 'I-GPE': 10,
    'B-LAW': 11, 'I-LAW': 12,
    'B-LOC': 13, 'I-LOC': 14,
    'B-MON': 15, 'I-MON': 16,
    'B-NOR': 17, 'I-NOR': 18,
    'B-ORD': 19, 'I-ORD': 20,
    'B-ORG': 21, 'I-ORG': 22,
    'B-PER': 23, 'I-PER': 24,
    'B-PRC': 25, 'I-PRC': 26,
    'B-PRD': 27, 'I-PRD': 28,
    'B-QTY': 29, 'I-QTY': 30,
    'B-REG': 31, 'I-REG': 32,
    'B-TIM': 33, 'I-TIM': 34,
    'B-WOA': 35, 'I-WOA': 36,
    'B-LAN': 37, 'I-LAN': 38
}

## Preprocessing

In [8]:
# use IndoBERT
tokenizer = AutoTokenizer.from_pretrained('indolem/indobert-base-uncased')

In [9]:
example = nergrit["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens[:10]

['[CLS]',
 'indonesia',
 'mengekspor',
 'produk',
 'industri',
 'skala',
 'besar',
 'ke',
 'amerika',
 'serikat']

Based on the documentation:

> This adds some special tokens [CLS] and [SEP] and the subword tokenization creates a mismatch between the input and labels. A single word corresponding to a single label may now be split into two subwords. You'll need to realign the tokens and labels by:

1. Mapping all tokens to their corresponding word with the word_ids method.
2. Assigning the label -100 to the special tokens [CLS] and [SEP] so they're ignored by the PyTorch loss function.
3. Only labeling the first token of a given word. Assign -100 to other subtokens from the same word.

So it is needed to realign the token and labels, and truncate the sequence if it is longer than the models maximum length

Based on the reference:

> It's more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

So, creating the data collator.

In [10]:
# Rename the 'labels' column to 'ner_tag'
nergrit["train"] = nergrit["train"].rename_column("ner_tag", "labels")
nergrit["validation"] = nergrit["validation"].rename_column("ner_tag", "labels")
nergrit["test"] = nergrit["test"].rename_column("ner_tag", "labels")

In [11]:
def tokenize_and_align_labels(dataset):
    tokenized_inputs = tokenizer(dataset["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, labels_per_example in enumerate(dataset["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                token_labels = [labels_dict[label] for label in labels_per_example]
                label_ids.append(token_labels[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [12]:
tokenized_nergrit = nergrit.map(tokenize_and_align_labels, batched=True) #processing in batch

Map:   0%|          | 0/12551 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/2402 [00:00<?, ? examples/s]

Map:   0%|          | 0/2526 [00:00<?, ? examples/s]

In [13]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="tf")

## Metrics Evaluation

I will be using just accuracy

In [14]:
seqeval = evaluate.load("seqeval")

In [15]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [unique_labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [unique_labels[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "accuracy": results["overall_accuracy"],
    }

## Fine-Tuning the Model

In [16]:
labels = list(labels_dict.keys())

id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in id2label.items()}


In [17]:
len(labels)

39

In [18]:
batch_size = 16
num_train_epochs = 3
num_train_steps = (len(tokenized_nergrit["train"]) // batch_size) * num_train_epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
    num_warmup_steps=0,
)

In [19]:
model = TFAutoModelForTokenClassification.from_pretrained(
    "indolem/indobert-base-uncased", 
    num_labels=39,          # set the num labels to 39
    id2label=id2label, 
    label2id=label2id, 
    from_pt=True
)

All PyTorch model weights were used when initializing TFBertForTokenClassification.

Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_nergrit["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_nergrit["validation"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [21]:
model.compile(optimizer=optimizer)

In [22]:
# create the callback for the model
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

push_to_hub_callback = PushToHubCallback(
    output_dir="indobert-base-uncased-finetuned-nergrit",  # change this based on the output repo desired
    tokenizer=tokenizer,
)

wandb_callback = wandb.keras.WandbCallback()

callbacks = [metric_callback, push_to_hub_callback, wandb_callback]

Cloning https://huggingface.co/apwic/indobert-base-uncased-finetuned-nergrit into local empty directory.


Download file tf_model.h5:   0%|          | 15.4k/420M [00:00<?, ?B/s]

Clean file tf_model.h5:   0%|          | 1.00k/420M [00:00<?, ?B/s]

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [23]:
model.fit(x=tf_train_set, 
          validation_data=tf_validation_set, 
          epochs=25,                 
          callbacks=callbacks)

Epoch 1/25
784/784 [==============================] - ETA: 0s - loss: 0.4507

wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/kaggle/working/wandb/run-20231025_081251-wrps5s5c/files/model-best)... Done. 10.4s


784/784 [==============================] - 262s 305ms/step - loss: 0.4507 - val_loss: 0.1933 - accuracy: 0.9437
Epoch 2/25
784/784 [==============================] - ETA: 0s - loss: 0.1708

wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/kaggle/working/wandb/run-20231025_081251-wrps5s5c/files/model-best)... Done. 5.1s


784/784 [==============================] - 222s 283ms/step - loss: 0.1708 - val_loss: 0.1795 - accuracy: 0.9471
Epoch 3/25
784/784 [==============================] - ETA: 0s - loss: 0.1295

wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/kaggle/working/wandb/run-20231025_081251-wrps5s5c/files/model-best)... Done. 5.0s


784/784 [==============================] - 222s 283ms/step - loss: 0.1295 - val_loss: 0.1784 - accuracy: 0.9483
Epoch 4/25
784/784 [==============================] - 167s 213ms/step - loss: 0.1169 - val_loss: 0.1784 - accuracy: 0.9483
Epoch 5/25
784/784 [==============================] - 165s 211ms/step - loss: 0.1172 - val_loss: 0.1784 - accuracy: 0.9483
Epoch 6/25
784/784 [==============================] - 164s 210ms/step - loss: 0.1180 - val_loss: 0.1784 - accuracy: 0.9483
Epoch 7/25
784/784 [==============================] - 165s 210ms/step - loss: 0.1176 - val_loss: 0.1784 - accuracy: 0.9483
Epoch 8/25
784/784 [==============================] - 165s 211ms/step - loss: 0.1172 - val_loss: 0.1784 - accuracy: 0.9483
Epoch 9/25
784/784 [==============================] - 165s 210ms/step - loss: 0.1168 - val_loss: 0.1784 - accuracy: 0.9483
Epoch 10/25
784/784 [==============================] - 165s 210ms/step - loss: 0.1174 - val_loss: 0.1784 - accuracy: 0.9483
Epoch 11/25
784/784 [=====

## Inferencing the Model

In [24]:
text = """Jakarta, Maret 1998
Di sebuah senja, di sebuah rumah susun di Jakarta, mahasiswa bernama Biru Laut disergap empat lelaki tak dikenal. Bersama kawan-kawannya, Daniel Tumbuan, Sunu Dyantoro, Alex Perazon, dia dibawa ke sebuah tempat yang tak dikenal. Berbulan-bulan mereka disekap, diinterogasi, dipukul, ditendang, digantung, dan disetrum agar bersedia menjawab satu pertanyaan penting: siapakah yang berdiri di balik gerakan aktivis dan mahasiswa saat itu.
Jakarta, Juni 1998
Keluarga Arya Wibisono, seperti biasa, pada hari Minggu sore memasak bersama, menyediakan makanan kesukaan Biru Laut. Sang ayah akan meletakkan satu piring untuk dirinya, satu piring untuk sang ibu, satu piring untuk Biru Laut, dan satu piring untuk si bungsu Asmara Jati. Mereka duduk menanti dan menanti. Tapi Biru Laut tak kunjung muncul.
Jakarta, 2000
Asmara Jati, adik Biru Laut, beserta Tim Komisi Orang Hilang yang dipimpin Aswin Pradana mencoba mencari jejak mereka yang hilang serta merekam dan mempelajari testimoni mereka yang kembali. Anjani, kekasih Laut, para orangtua dan istri aktivis yang hilang menuntut kejelasan tentang anggota keluarga mereka. Sementara Biru Laut, dari dasar laut yang sunyi bercerita kepada kita, kepada dunia tentang apa yang terjadi pada dirinya dan kawan-kawannya.
Laut Bercerita, novel terbaru Leila S. Chudori, bertutur tentang kisah keluarga yang kehilangan, sekumpulan sahabat yang merasakan kekosongan di dada, sekelompok orang yang gemar menyiksa dan lancar berkhianat, sejumlah keluarga yang mencari kejelasan akan anaknya, dan tentang cinta yang tak akan luntur."""

In [25]:
# use pipeline to easen inferencing the model
classifier = pipeline("ner", model="apwic/indobert-base-uncased-finetuned-nergrit")

Some layers from the model checkpoint at apwic/indobert-base-uncased-finetuned-nergrit were not used when initializing TFBertForTokenClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForTokenClassification were initialized from the model checkpoint at apwic/indobert-base-uncased-finetuned-nergrit.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


In [26]:
classified_text = classifier(text)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [27]:
pd.DataFrame(classified_text[:10])

,entity,score,index,word,start,end
0,B-GPE,0.990143,1,jakarta,0,7
1,B-DAT,0.986492,3,maret,9,14
2,I-DAT,0.993397,4,1998,15,19
3,B-GPE,0.994223,14,jakarta,62,69
4,B-PER,0.461920,18,biru,89,93
5,I-PER,0.690567,19,laut,94,98
6,B-CRD,0.914409,22,empat,108,113
7,B-PER,0.995100,32,daniel,158,164
8,I-PER,0.993357,33,tum,165,168
9,I-PER,0.991130,34,##buan,168,172


In [28]:
def visualize_entities_html(text, classified_text):
    # Create a color mapping for each entity type
    color_map = {
        'GPE': 'yellow',
        'DAT': 'blue',
        'PER': 'green',
        'CRD': 'red',
        'EVT': '#FFA500',  # orange
        'FAC': '#FFC0CB',  # pink
        'LAW': '#FFD700',  # gold
        'LOC': '#ADFF2F',  # greenyellow
        'MON': '#FA8072',  # salmon
        'NOR': '#9370DB',  # mediumpurple
        'ORD': '#7B68EE',  # mediumslateblue
        'ORG': '#6A5ACD',  # slateblue
        'PRC': '#FF69B4',  # hotpink
        'PRD': '#D2B48C',  # tan
        'QTY': '#FF6347',  # tomato
        'REG': '#DB7093',  # palevioletred
        'TIM': '#EEE8AA',  # palegoldenrod
        'WOA': '#F08080',  # lightcoral
        'LAN': '#BDB76B'   # darkkhaki
    }

    
    # Sort classified_text by start index
    classified_text.sort(key=lambda x: x['start'])

    html_output = text
    shift = 0
    
    for entity in classified_text:
        word = entity['word']
        start = entity['start'] + shift
        end = entity['end'] + shift
        entity_type = entity['entity'].split('-')[-1]  # Extracting main entity type e.g., 'B-GPE' -> 'GPE'
        color = color_map.get(entity_type, 'grey')  # Default to grey if entity type is not in our map
        
        # Wrap the word in a span with background color
        span = f"<span style='background-color: {color}'>{word}</span>"
        
        # Replace the word in the text with the highlighted word
        html_output = html_output[:start] + span + html_output[end:]
        
        # Adjust shift based on the added HTML tags
        shift += len(span) - (end - start)
    
    display(HTML(html_output))

visualize_entities_html(text, classified_text)